
# Aggregating vegetative propogation group data
___

Load the data.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as sf
import matplotlib.pyplot as plt

In [2]:
from pi_for_temperature_notebook import *
options = get_veg_prop_args()

In [3]:
data = pd.read_csv("variant-calls_vegprop-expt_2020-04-07_tables.csv",header=0,index_col=0)

In [39]:
data_both_present = data.dropna(subset=['ADReplicateA','ADReplicateB'])
filtered_data = pd.concat([data_both_present.loc[data_both_present.freqPropReplicateA >= 0.03],
                           data_both_present.loc[data_both_present.freqPropReplicateB >= 0.03]]).drop_duplicates()

In [40]:
#lineage: set to 0 then assign each unique lineage to an unique integer
filtered_data['lineage_factor'] = 0
j = 0
for lineage in filtered_data.lineage.unique():
    filtered_data.loc[filtered_data.lineage == lineage,['lineage_factor']] = j
    j += 1

#species: Set ACMV = 0 and EACMCV = 1
filtered_data['species'] = 0
filtered_data.loc[filtered_data.chrom == 'EACMCV DNA-A',['species']] = 1
filtered_data.loc[filtered_data.chrom == 'EACMCV DNA-B',['species']] = 1

#segment: Set DNA-A = 0 and DNA-B = 1
filtered_data['segment'] = 0
filtered_data.loc[filtered_data.chrom == 'ACMV DNA-B',['segment']] = 1
filtered_data.loc[filtered_data.chrom == 'EACMCV DNA-B',['segment']] = 1

#SEGs treatment: Virus Only:0, SEGS-1:1,SEGS-2:2
filtered_data['SEGs_treatment'] = 0
filtered_data.loc[filtered_data.segsTreatment2 == 'SEGS-1','SEGs_treatment'] = 1
filtered_data.loc[filtered_data.segsTreatment2 == 'SEGS-2','SEGs_treatment'] = 2

In [ ]:
agg_data = average_groups(filtered_data,['species','segment','pos','alt','passage','ref','lineage_factor','SEGs_treatment'])

agg_data.reset_index(inplace=True)
agg_data['pos'] = agg_data.pos.astype(int)

In [ ]:
options['frequency'] = 'freqPropMeanNoNA'

pi_df = get_group_pis(agg_data,
                     options=options,
                     group_by=['species','segment','passage','lineage_factor','SEGs_treatment'])

lm = sf.ols('pi ~ passage + C(SEGs_treatment) + (C(species)/C(segment)) + C(lineage_factor)',data=pi_df).fit(cov_type='HC1')
print("Least squares summary:")
print(lm.summary())

print("\nAnova table:")
table = sm.stats.anova_lm(lm)
print(table)

print()
for ind in table.index:
    
    print("{} explains \t {:.2%} of variance".format(ind,table.loc[ind,'sum_sq'] / table.sum_sq.sum()))

In [ ]:
p = pi_df.boxplot(column='pi',
             by=['lineage_factor'],
             patch_artist=True,
             grid=False,
             return_type='dict')

plt.ylabel('pi',fontsize=16)
plt.xlabel('lineage',fontsize=16)
ylim = plt.ylim()

plt.xticks(fontsize=14,ha='right')
plt.suptitle('')
plt.title('')

ylim = plt.ylim()

#plt.vlines(x=[3.5,6.5,9.5,12.5,15.5,18.5],ymin=ylim[0],ymax=ylim[1])
#plt.ylim(ylim)

for median in p[0]['medians']:
    median.set_color('k')

In [ ]:
p = pi_df.boxplot(column='pi',
             by=['species','segment'],
             patch_artist=True,
             grid=False,
             return_type='dict',
             rot=30)

plt.ylabel('pi',fontsize=16)
plt.xlabel('',fontsize=16)
ylim = plt.ylim()

plt.vlines(x=[2.5],ymin=ylim[0],ymax=ylim[1])
plt.ylim(ylim)

plt.xticks([1,2,3,4],
           ['ACMV DNA-A','ACMV DNA-B','EACMCV DNA-A','EACMCV DNA-B'],
           fontsize=14,
           ha='right')
plt.suptitle('')
plt.title('')

for median in p[0]['medians']:
    median.set_color('k')

In [ ]:
agg_data = average_groups(filtered_data,
                          ['species','segment','pos','alt','passage','ref','SEGs_treatment'])

agg_data.reset_index(inplace=True)
agg_data['pos'] = agg_data.pos.astype(int)

options['frequency'] = 'freqPropMeanNoNA'

#print("Number of variants")

In [ ]:
pi_df = get_group_pis(agg_data,
                     options=options,
                     group_by=['species','segment','passage','SEGs_treatment'])

lm = sf.ols('pi ~ passage + C(SEGs_treatment) + C(species)/C(segment)',data=pi_df).fit(cov_type='HC1')
print("Least squares summary:")
print(lm.summary())

print("\nAnova table:")
table = sm.stats.anova_lm(lm)
print(table)
    
print("\nEffect\t\t% var explained")
for ind in table.index:
    
    print("{} \t\t {:.2%}".format(ind,table.loc[ind,'sum_sq'] / table.sum_sq.sum()))    

In [ ]:
p = pi_df.boxplot(column='pi',
             by=['passage','SEGs_treatment'],
             patch_artist=True,
             grid=False,
             return_type='dict',
             rot=30)

plt.ylabel('pi',fontsize=16)
plt.xlabel('(passage,SEGs_treatment)',fontsize=16)
plt.xticks(ha='right')
ylim = plt.ylim()

plt.vlines(x=[3.5,6.5,9.5,12.5,15.5,18.5],ymin=ylim[0],ymax=ylim[1])
plt.ylim(ylim)

plt.xticks(fontsize=14)
plt.suptitle('')
plt.title('')

for median in p[0]['medians']:
    median.set_color('k')

In [53]:
agg_data = average_groups(filtered_data,
                          ['species','segment','pos','alt','passage','ref'])

agg_data.reset_index(inplace=True)
agg_data['pos'] = agg_data.pos.astype(int)

options['frequency'] = 'freqPropMeanNoNA'

In [54]:
pi_df = get_group_pis(agg_data,
                     options=options,
                     group_by=['species','segment','passage'])

"""
lm = sf.ols('pi ~ passage + C(species)/C(segment)',data=pi_df).fit(cov_type='HC1')
print("Least squares summary:")
print(lm.summary())

print("\nAnova table:")
table = sm.stats.anova_lm(lm)
print(table)
    
print("\nEffect\t\t% var explained")
for ind in table.index:
    
    print("{} \t\t {:.2%}".format(ind,table.loc[ind,'sum_sq'] / table.sum_sq.sum()))
"""    

'\nlm = sf.ols(\'pi ~ passage + C(species)/C(segment)\',data=pi_df).fit(cov_type=\'HC1\')\nprint("Least squares summary:")\nprint(lm.summary())\n\nprint("\nAnova table:")\ntable = sm.stats.anova_lm(lm)\nprint(table)\n    \nprint("\nEffect\t\t% var explained")\nfor ind in table.index:\n    \n    print("{} \t\t {:.2%}".format(ind,table.loc[ind,\'sum_sq\'] / table.sum_sq.sum()))\n'

In [44]:
import statsmodels.api as sm
import statsmodels.formula.api as sf
from linearmodels import PanelOLS

In [55]:
pi_df = pi_df.set_index(['passage','segment'])


In [57]:
exog = sm.add_constant(pi_df[['species']])

mod = PanelOLS(pi_df.pi,exog, entity_effects=True)

panel_res = mod.fit(cov_type='robust')

print(panel_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                     pi   R-squared:                        0.1208
Estimator:                   PanelOLS   R-squared (Between):              0.0000
No. Observations:                  28   R-squared (Within):               0.1208
Date:                Thu, Jun 18 2020   R-squared (Overall):              0.0544
Time:                        19:23:54   Log-likelihood                   -98.250
Cov. Estimator:                Robust                                           
                                        F-statistic:                      2.7484
Entities:                           7   P-value                           0.1130
Avg Obs:                       4.0000   Distribution:                    F(1,20)
Min Obs:                       4.0000                                           
Max Obs:                       4.0000   F-statistic (robust):             2.7484
                            